# Tutorial

In [ ]:
using DiffEqFlux,DifferentialEquations, Optimization, OptimizationOptimJL, Random, Plots, CSV, Lux, DataFrames
using DataDrivenDiffEq, ModelingToolkit, LinearAlgebra, DiffEqSensitivity, Zygote, Optim, CSV, Lux, Pkg, Flux
gr()
Pkg.status()

### ODE data for simmulation

In [ ]:
rng = Random.default_rng()
u0 = Float32[2.0; 0.0]
datasize = 30
tspan = (0.0f0, 1.5f0)
tsteps = range(tspan[1], tspan[2], length = datasize)

function trueODEfunc(du, u, p, t)
    true_A = [-0.1 2.0; -2.0 -0.1]
    du .= ((u.^3)'true_A)'
end

prob_trueode = ODEProblem(trueODEfunc, u0, tspan)
ode_data = Array(solve(prob_trueode, Tsit5(), saveat = tsteps))

### This is an Neural ODE

In [ ]:
dudt2 = Lux.Chain(x -> x.^3,
                  Lux.Dense(2, 50, tanh),
                  Lux.Dense(50, 2))
p, st = Lux.setup(rng, dudt2)
prob_neuralode = NeuralODE(dudt2, tspan, Tsit5(), saveat = tsteps)

In [ ]:
function predict_neuralode(p)
  Array(prob_neuralode(u0, p, st)[1])
end

function loss_neuralode(p)
    pred = predict_neuralode(p)
    loss = sum(abs2, ode_data .- pred)
    return loss, pred
end

# Do not plot by default for the documentation
# Users should change doplot=true to see the plots callbacks
callback = function (p, l, pred; doplot = true)
  println(l)
  # plot current prediction against data
  if doplot
    plt = scatter(tsteps, ode_data[1,:], label = "data")
    scatter!(plt, tsteps, pred[1,:], label = "prediction")
    display(plot(plt))
  end
  return false
end

pinit = Lux.ComponentArray(p)
callback(pinit, loss_neuralode(pinit)...; doplot=true)

# use Optimization.jl to solve the problem
adtype = Optimization.AutoZygote()

optf = Optimization.OptimizationFunction((x, p) -> loss_neuralode(x), adtype)
optprob = Optimization.OptimizationProblem(optf, pinit)

result_neuralode = Optimization.solve(optprob,
                                       ADAM(0.05),
                                       callback = callback,
                                       maxiters = 300)

optprob2 = remake(optprob,u0 = result_neuralode.u)

result_neuralode2 = Optimization.solve(optprob2,
                                        Optim.BFGS(initial_stepnorm=0.01),
                                        callback=callback,
                                        allow_f_increases = false)

callback(result_neuralode2.u, loss_neuralode(result_neuralode2.u)...; doplot=true)

# Our data

In [ ]:
data_dir = "/Users/adrocampos/covid19/synth_data/"
regions = ["2", "3", "5", "10", "15", "20", "30"][1]
mobility_type = ["inv_dist", "border", "neighbor"][2]
initially_recovered = false

In [ ]:
positions = CSV.File(data_dir * "positions_" * regions * "_regions.csv")
positions = DataFrame(positions)

In [ ]:
file = "1"

In [ ]:
csv_reader = CSV.File(data_dir * "SIR_" * regions * "_regions_" * mobility_type * "_" * file * ".csv")
df = DataFrame(csv_reader)

In [ ]:
index = range(1,stop=5001,step=50)
df = df[index,:]

In [ ]:
X = Matrix(df[:, [:S1, :I1, :R1]])'
t = df.t

In [ ]:
tspan=(t[begin], t[end])
tsteps = range(tspan[1], tspan[2], length = size(t)[1])

u0 = X[:,1]

### Neural ODE for our data

In [ ]:
rng = Random.default_rng()

dudt2 = Lux.Chain(Lux.Dense(3, 50, tanh),
                  Lux.Dense(50, 50, tanh),
                  Lux.Dense(50, 3))

p, st = Lux.setup(rng, dudt2)
prob_neuralode = NeuralODE(dudt2, tspan, Tsit5(), saveat = tsteps)

In [ ]:
function predict_neuralode(p)
  Array(prob_neuralode(u0, p, st)[1])
end

function loss_neuralode(p)
    pred = predict_neuralode(p)
    loss = sum(abs2, X .- pred)
    return loss, pred
end

# Do not plot by default for the documentation
# Users should change doplot=true to see the plots callbacks
callback = function (p, l, pred; doplot = true)
  println(l)
  # plot current prediction against data
  if doplot
        
        
#     plt = scatter(tsteps, X[1,:], label = "data")
#     scatter!(plt, tsteps, pred[1,:], label = "prediction")
        
    pS = scatter(tsteps, X[1,:], label = "data S")
    scatter!(pS, tsteps, pred[1,:], label = "prediction S")
        
    pI = scatter(tsteps, X[2,:], label = "data I")
    scatter!(pI, tsteps, pred[2,:], label = "prediction I")
        
    pR = scatter(tsteps, X[3,:], label = "data R")
    scatter!(pR, tsteps, pred[3,:], label = "prediction R")
   
    display(plot(pS, pI, pR, layout = (3,1)))
            
        
  end
  return false
end

In [ ]:
pinit = Lux.ComponentArray(p)
callback(pinit, loss_neuralode(pinit)...; doplot=true)

# use Optimization.jl to solve the problem
adtype = Optimization.AutoZygote()

optf = Optimization.OptimizationFunction((x, p) -> loss_neuralode(x), adtype)
optprob = Optimization.OptimizationProblem(optf, pinit)

result_neuralode = Optimization.solve(optprob,
                                       ADAM(0.05),
                                       callback = callback,
                                       maxiters = 300)



optprob2 = remake(optprob, u0 = result_neuralode.u)

result_neuralode2 = Optimization.solve(optprob2,
                                        Optim.BFGS(initial_stepnorm=0.01),
                                        callback=callback,
                                        allow_f_increases = false)

callback(result_neuralode2.u, loss_neuralode(result_neuralode2.u)...; doplot=true)

## Universal Differential Equations

In [ ]:
# Define the problem
prob_nn = ODEProblem(ude_dynamics!,Xₙ[:, 1], tspan, p)

## Function to train the network
# Define a predictor
function predict(θ, X = Xₙ[:,1], T = t)
    Array(solve(prob_nn, Vern7(), u0 = X, p=θ,
                tspan = (T[1], T[end]), saveat = T,
                abstol=1e-6, reltol=1e-6,
                sensealg = ForwardDiffSensitivity()
                ))
end


# Define parameters for Multiple Shooting
group_size = 5
continuity_term = 200.0f0

function loss(data, pred)
	return sum(abs2, data - pred)
end

function shooting_loss(p)
    return multiple_shoot(p, Xₙ, t, prob_nn, loss, Vern7(),
                          group_size; continuity_term)
end

function loss(θ)
    X̂ = predict(θ)
    sum(abs2, Xₙ - X̂) / size(Xₙ, 2) + convert(eltype(θ), 1e-3)*sum(abs2, θ[3:end]) ./ length(θ[3:end])
end

# Container to track the losses
losses = Float32[]

# Callback to show the loss during training
callback(θ,args...) = begin
	l = loss(θ) # Equivalent L2 loss
    push!(losses, l)
    if length(losses)%5==0
        println("Current loss after $(length(losses)) iterations: $(losses[end])")
    end
    false
end

## Training -> First shooting / batching to get a rough estimate

# First train with ADAM for better convergence -> move the parameters into a
# favourable starting positing for BFGS
res1 = DiffEqFlux.sciml_train(shooting_loss, p, ADAM(0.1f0), cb=callback, maxiters = 100)
println("Training loss after $(length(losses)) iterations: $(losses[end])")
# Train with BFGS to achieve partial fit of the data
res2 = DiffEqFlux.sciml_train(shooting_loss, res1.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters = 500)
println("Training loss after $(length(losses)) iterations: $(losses[end])")
# Full L2-Loss for full prediction
res3 = DiffEqFlux.sciml_train(loss, res2.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters = 10000)
println("Final training loss after $(length(losses)) iterations: $(losses[end])")

In [ ]:
ann = FastChain(FastDense(3, 50, tanh),
                FastDense(50, 50, tanh),
                FastDense(50, 1))



## Firs the parameters for Beta, gama und N, then the weigths. 

# Get the initial parameters, first two is linear birth / decay of prey and predator
p = [rand(Float32,2); 2000; initial_params(ann)]

In [ ]:
# rng = Random.default_rng()

# ann = Lux.Chain(Lux.Dense(3, 50, tanh),
#                   Lux.Dense(50, 50, tanh),
#                   Lux.Dense(50, 1))

# pinit, st = Lux.setup(rng, ann)

# p = [rand(Float32,2); 2000; pinit]


In [ ]:
size(p)

In [ ]:
# Define the hybrid model
function ude_dynamics!(du,u, p, t)
    û = U(u, p[3:end]) # Network prediction
    # We assume a linear birth rate for the prey
    du[1] = p[1]*u[1] + û[1]
    # We assume a linear decay rate for the predator
    du[2] = -p[2]*u[2] + û[2]
end

In [ ]:
function dudt_(du, u, p, t)
    
    S, I, R = u
    β, γ, N  = p[1:3]

    z  = ann(u, p[3:end])
    dS = -β * S * I/N - z[1]  # susceptible
    dI =  β * S * I/N - γ*I - z[1] # infected
    dR =  γ * I

    du[1] = dS
    du[2] = dI
    du[3] = dR

end



# Define the problem
prob_UODE = ODEProblem(dudt_, u0, tspan, p) ##prob_neuralode

In [ ]:

# Define parameters for Multiple Shooting
group_size = 5
continuity_term = 200.0f0

# function loss(data, pred)
# 	return sum(abs2, data - pred)
# end

# function shooting_loss(p)
#     return multiple_shoot(p, X, t, prob_UODE, loss, Vern7(),
#                           group_size; continuity_term)
# end

function loss(θ)
    X̂ = predict(θ)
    sum(abs2, X - X̂) / size(X, 2) + convert(eltype(θ), 1e-3)*sum(abs2, θ[3:end]) ./ length(θ[3:end])
    return loss, pred
end



# Do not plot by default for the documentation
# Users should change doplot=true to see the plots callbacks
callback = function (p, l, pred; doplot = true)
  println(l)
  # plot current prediction against data
  if doplot
        
        
#     plt = scatter(tsteps, X[1,:], label = "data")
#     scatter!(plt, tsteps, pred[1,:], label = "prediction")
        
    pS = scatter(tsteps, X[1,:], label = "data S")
    scatter!(pS, tsteps, pred[1,:], label = "prediction S")
        
    pI = scatter(tsteps, X[2,:], label = "data I")
    scatter!(pI, tsteps, pred[2,:], label = "prediction I")
        
    pR = scatter(tsteps, X[3,:], label = "data R")
    scatter!(pR, tsteps, pred[3,:], label = "prediction R")
   
    display(plot(pS, pI, pR, layout = (3,1)))
            
        
  end
  return false
end

# Container to track the losses
# losses = Float32[]

# Callback to show the loss during training
# callback(θ,args...) = begin
# 	l = loss(θ) # Equivalent L2 loss
#     push!(losses, l)
#     if length(losses)%5==0
#         println("Current loss after $(length(losses)) iterations: $(losses[end])")
#     end
#     false
# end

## Training -> First shooting / batching to get a rough estimate

# First train with ADAM for better convergence -> move the parameters into a
# favourable starting positing for BFGS


# res1 = DiffEqFlux.sciml_train(shooting_loss, p, Adam(0.1f0) , cb=callback, maxiters = 1)
# println("Training loss after $(length(losses)) iterations: $(losses[end])")
# Train with BFGS to achieve partial fit of the data
# res2 = DiffEqFlux.sciml_train(shooting_loss, res1.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters = 500)
# println("Training loss after $(length(losses)) iterations: $(losses[end])")
# # Full L2-Loss for full prediction
# res3 = DiffEqFlux.sciml_train(loss, res2.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters = 10000)
# println("Final training loss after $(length(losses)) iterations: $(losses[end])")

In [ ]:
# pinit = Lux.ComponentArray(p)
# callback(pinit, loss_neuralode(pinit)...; doplot=true)

# # use Optimization.jl to solve the problem
# adtype = Optimization.AutoZygote()

# optf = Optimization.OptimizationFunction((x, p) -> loss_neuralode(x), adtype)
# optprob = Optimization.OptimizationProblem(optf, pinit)

# result_neuralode = Optimization.solve(optprob,
#                                        ADAM(0.05),
#                                        callback = callback,
#                                        maxiters = 300)

In [ ]:
callback = function (p, l, pred; doplot = true)
  println(l)
#   plot current prediction against data
  if doplot
        
        
#     plt = scatter(tsteps, X[1,:], label = "data")
#     scatter!(plt, tsteps, pred[1,:], label = "prediction")
        
    pS = scatter(tsteps, X[1,:], label = "data S")
    scatter!(pS, tsteps, pred[1,:], label = "prediction S")
        
    pI = scatter(tsteps, X[2,:], label = "data I")
    scatter!(pI, tsteps, pred[2,:], label = "prediction I")
        
    pR = scatter(tsteps, X[3,:], label = "data R")
    scatter!(pR, tsteps, pred[3,:], label = "prediction R")
   
    display(plot(pS, pI, pR, layout = (3,1)))
            
        
  end
  return false
end

In [ ]:
# pinit = Lux.ComponentArray(p)

# use Optimization.jl to solve the problem
adtype = Optimization.AutoZygote()
optf    = Optimization.OptimizationFunction((x,p) -> loss(x), adtype)
optprob = Optimization.OptimizationProblem(optf, p)

result_neuralode = Optimization.solve(prob_UODE, Tsit5(), callback=callback, maxiters = 300)

In [ ]:
## How to plot the loss using Optimization?

In [ ]:
pred[:,1]

In [ ]:
result_neuralode.t

In [ ]:
pred = reduce(hcat,result_neuralode.u)
t = result_neuralode.t

tspan = (t[begin], t[end])

steps = range(1,87)


# println(tsteps)

# pred = predict(p_trained, X[:,1], tsample)

pS = scatter(pred[1,:], label = "data S")

# # # # scatter!(pS, tsteps, pred[1,:], label = "prediction S")

pI = scatter(pred[2,:], label = "data I")
scatter!(pI, tsteps, pred[2,:], label = "prediction I")
        
# # # # pR = scatter(tsteps, X[3,:], label = "data R")b
# # # # scatter!(pR, tsteps, pred[3,:], label = "prediction R")
   
display(plot(pS, pI, layout = (3,1)))

In [ ]:
t

In [ ]:
## Analysis of the trained network
# Interpolate the solution



In [ ]:
result_neuralode

In [ ]:
# use Optimization.jl to solve the problem
adtype = Optimization.AutoZygote()

optf = Optimization.OptimizationFunction(loss, adtype)
optprob = Optimization.OptimizationProblem(optf, p)

result_neuralode = Optimization.solve(prob_UODE,
                                       Tsit5(),
#                                        callback = callback,
                                       maxiters = 300)

In [ ]:

function predict(θ)
    Array(concrete_solve(prob_UODE, Vern7(), u0, θ, saveat = t))
end

# No regularisation right now
function loss(θ)
    pred = predict(θ)
    sum(abs2, X .- pred), pred # + 1e-5*sum(sum.(abs, params(ann)))
end

loss(p)

const losses = []
callback(θ,l,pred) = begin
    push!(losses, l)
#     if length(losses)%50==0
#         println(losses[end])
#     end
#     false
end

res1_uode = DiffEqFlux.sciml_train(loss, p, ADAM(0.01), cb=callback, maxiters = 5)
# res2_uode = DiffEqFlux.sciml_train(loss, res1_uode.minimizer, BFGS(initial_stepnorm=0.01), cb=callback, maxiters = 10000)

In [ ]:
# function predict(θ, X=X[:,1], T=t)
#     Array(solve(prob_UODE, Vern7(), u0=X, p=θ, tspan=tspan, saveat=T))
#     end
    
    
    
# function loss(data, pred)
# 	return sum(abs2, data - pred)
# end

    
    
function loss(θ)
    X̂ = predict(θ)
    sum(abs2, Xₙ - X̂) / size(Xₙ, 2) + convert(eltype(θ), 1e-3)*sum(abs2, θ[3:end]) ./ length(θ[3:end])
end

In [ ]:
# use Optimization.jl to solve the problem
adtype = Optimization.AutoZygote()

optf = Optimization.OptimizationFunction((x, p) -> loss(x), adtype)
optprob = Optimization.OptimizationProblem(optf, pinit)

result_neuralode = Optimization.solve(prob_UODE,
                                       Adam(0.05),
#                                        callback = callback,
                                       maxiters = 300)

In [ ]:
result_neuralode

In [ ]:
# Container to track the losses
losses = Float32[]

# Callback to show the loss during training
callback(θ,args...) = begin
	l = loss(θ) # Equivalent L2 loss
    print(l)
    push!(losses, l)
    if length(losses)%5==0
        println("Current loss after $(length(losses)) iterations: $(losses[end])")
    end
    false
end




In [ ]:
## Training -> First shooting / batching to get a rough estimate

# First train with ADAM for better convergence -> move the parameters into a
# favourable starting positing for BFGS
res1 = DiffEqFlux.sciml_train(shooting_loss, p, ADAM(0.1f0), cb=callback, maxiters = 100)
println("Training loss after $(length(losses)) iterations: $(losses[end])")
# Train with BFGS to achieve partial fit of the data
res2 = DiffEqFlux.sciml_train(shooting_loss, res1.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters = 500)
println("Training loss after $(length(losses)) iterations: $(losses[end])")
# Full L2-Loss for full prediction
res3 = DiffEqFlux.sciml_train(loss, res2.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters = 10000)
println("Final training loss after $(length(losses)) iterations: $(losses[end])")



In [ ]:
res1 = DiffEqFlux.sciml_train(loss, p, ADAM(0.1f0), cb=callback, maxiters = 100)
println("Training loss after $(length(losses)) iterations: $(losses[end])")



In [ ]:
p_trained = res1.minimizer